# 23.1 多智能体强化学习入门
- 基本概念
- IPPO 算法

> 之前介绍的算法都是 **单智能体** 强化学习算法，其基本假设是动态环境是 **稳态的（stationary）**，即 **状态转移概率** 和 **奖励函数** 不变

> 但在现实中，许多系统并非由单一主体组成，而是由 **多个智能体** 构成。在一些任务中，智能体可能需要合作以共同完成任务（如机器人协作、团队对抗游戏），也可能需要竞争以实现对抗模拟（如博弈论中的零和游戏）此时就要求环境中有多个智能体进行交互和学习，这样的任务被称为 **多智能体强化学习（multi-agent reinforcement learning，MARL）**

> **MARL** 中的每个智能体在和环境交互的同时也在和 **其他智能体** 进行直接或者间接的**交互**，因此在每个智能体的视角下，环境是 **非稳态的（non-stationary）**：对于一个智能体而言，即使在 **相同的状态** 下采取 **相同的动作**，得到的状态转移和奖励信号的分布可能在**不断改变**
> 所以多智能体的协作，本身会造就：
- 训练的环境更为复杂
> 其次:
- 智能体目标可能是不同的，不同智能体需要最大化自己的利益
- 训练难度更高，可能需要大规模分布式训练来提高效率

***参考文献：***
1. [多智能体深度强化学习的若干关键科学问题](https://kns.cnki.net/kcms2/article/abstract?v=hEVkP-djbGzQFZYseuDP5znRnCKidSEjzwYYgqH7xXiRLM90zl4RCyS7NqBJ9ADgnVnqy7EFO_RTQU-IvB4q7kgt0X64dkA9SwmnGm5AUMOwU9B38ZnNOfeVxU0gDSmgGyTGkNXHoPWIJwW0nmBKe-o19dokZVne9ebzRraoD1_TqMx5EDX-xQ==&uniplatform=NZKPT&language=CHS)
2. [A Survey and Critique of Multiagent Deep Reinforcement Learning](https://arxiv.org/abs/1810.05587)

***多智能体环境表示：***
> 可以用 **一个元组$(N,\mathcal{S},\mathcal{A},\mathcal{R},P)$** 来表示 **一个多智能体环境**
- 其中$N$是智能体的数目
- $\mathcal{S}=S_1\times\cdots\times S_N$ 是 所有智能体的状态集合
- $\mathcal{A}=A_1\times\cdots\times A_N$ 是 所有智能体的动作集合
- $\mathcal{R}=r_1\times\cdots\times r_N$ 是 所有智能体奖励函数的集合
- $P$是环境的状态转移概率

## 23.1.1 多智能体强化学习的基本求解范式
> 如果只是基于之前已经熟悉的 **单智能体算法**，拓展多智能体的概念，那么 **多智能体强化学习算法** 主要分两种思路：

> 1. **完全中心化（fully centralized）方法** ：将多个智能体进行决策当作一个 **超级智能体** 在进行决策（*所有智能体的状态聚合在一起当作一个全局的超级状态；所有智能体的动作连起来作为一个联合动作*）。
- 优点：由于已经知道了所有智能体的状态和动作，因此对这个 **超级智能体** 来说，**环境依旧是稳态**的，一些**单智能体的算法的收敛性**依旧可以得到保证。
- 缺点：样的做法不能很好地扩展到智能体数量很多或者环境很大的情况，因为这时候将所有的信息简单暴力地拼在一起会**导致维度爆炸**，**训练复杂度巨幅提升**的问题往往不可解决。

> 2. **完全去中心化（fully decentralized）方法** ：与完全中心化方法相反的范式便是假设 **每个智能体都在自身的环境中独立地进行学习，不考虑其他智能体的改变**。完全去中心化方法直接对 **每个智能体用一个单智能体强化学习算法来学习**。
- 缺点：环境是非稳态的，智能体之间没有信息共享，训练的收敛性不能得到保证。
- 优点：随着智能体数量的增加有比较好的扩展性，不会遇到维度灾难而导致训练不能进行下去。


## 23.1.2 完全去中心化 算法 IPPO
> 此类算法被称为 **独立学习（Independent Learning）**。由于对于每个智能体使用单智能体算法 **PPO（PPO-截断版本）** 进行训练，因此这个算法叫作 **独立 PPO（Independent PPO，IPPO）算法**。

***算法流程：***

对于$N$个智能体，**初始化** 每个智能体各自的策略以及价值函数
for 训练轮数 $k= 0, 1, 2\ldots$ do

- 所有智能体在环境中交互 **分别获得各自的一条轨迹数据**
- 对每个智能体，基于当前的价值函数用 GAE 计算优势函数的估计
- 对每个智能体，通过最大化其 PPO-截断的目标来 **更新其策略**
- 对每个智能体，通过均方误差损失函数 **优化其价值函数**

end for


## 23.1.3 Combat 环境说明
> **Combat 环境** 是 **ma_gym库**（[项目链接_GitHub](https://github.com/boyu-ai/ma-gym)） 中的一个多智能体环境：
![Combat-v0](Illustrations/Combat-v0.gif)


In [4]:
from ma_gym.envs.combat.combat import Combat

ModuleNotFoundError: No module named 'gym'

## 23.1.4 IPPO 代码实践

在 Combat 环境中，共享了两个智能体之间的策略，以达到更好的效果。但这仅限于多个智能体同质的情况，若它们的状态空间或动作空间不一致，那便无法进行策略共享。